# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [1]:
import os
import glob,time
import subprocess as sp
import numpy as np

In [2]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/run_scripts


In [4]:
code_dirs={'128':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code',
         '512':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/512_sqr'}

install_dirs={'off':'/global/cfs/cdirs/m3363/lbann/tom_lbann_install_20210126' # LBANN_DETERMINISTIC=OFF
             ,'on':'/global/cfs/cdirs/m3363/lbann/tom_lbann_install_deterministic_20201201' # LBANN_DETERMINISTIC=ON
             }

In [15]:
### Default dictionary
img_size=128

dict_pars={'epochs':10,
           'seed':3672323, 'nodes':1, 'procs':8,'job_name':'scale0.1_deterministic_off',
#            'spec_loss':'',
           'spec_loss':'--spec_loss',
           'mcr':'',
           'lambda_spec':0.1,'learn_rate':0.002,
           'deterministic':'off', # on or off
          'time':'2:00:00', 'batchsize':256, 'step_interval':10}

# dict_pars['job_name']=dict_pars['job_name']+dict_pars['nodes']
dict_pars['code_dir']=code_dirs[str(img_size)]
dict_pars['install_loc']=install_dirs[dict_pars['deterministic']]
print(dict_pars)

{'epochs': 10, 'seed': 3672323, 'nodes': 1, 'procs': 8, 'job_name': 'scale0.1_deterministic_off', 'spec_loss': '--spec_loss', 'mcr': '', 'lambda_spec': 0.1, 'learn_rate': 0.002, 'deterministic': 'off', 'time': '2:00:00', 'batchsize': 256, 'step_interval': 10, 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code', 'install_loc': '/global/cfs/cdirs/m3363/lbann/tom_lbann_install_20210126'}


In [16]:
# ### Default dictionary
# img_size=512

# dict_pars={'epochs':30,
#            'seed':3672323, 'nodes':1, 'procs':8,'job_name':'test_512',
# #            'spec_loss':'',
#            'spec_loss':'--spec_loss',
#            'mcr':'',
#            'learn_rate':0.0002,
#           'time':'2:00:00', 'batchsize':64, 'step_interval':10}

# # dict_pars['job_name']=dict_pars['job_name']+dict_pars['nodes']
# dict_pars['code_dir']=code_dirs[str(img_size)]
# print(dict_pars)

In [17]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes={nodes}
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8 
#SBATCH --cpus-per-task=8 
#SBATCH --gpus-per-task=1
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

### Initial setup
module purge
module load cgpu
module load modules/3.2.11.4 gcc/8.3.0 cuda/10.2.89 mvapich2/2.3.2 cmake/3.14.4 python/3.7-anaconda-2019.10

module use {install_loc}/etc/modulefiles
module load lbann

export MKL_THREADING_LAYER=GNU

#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git
export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/tom_spack
$SPACK_ROOT/share/spack/setup-env.sh
export MV2_ENABLE_AFFINITY=0
export MV2_USE_CUDA=1
export IBV_FORK_SAFE=1

### Run the main code
code_dir={code_dir}

python $code_dir/train_exagan.py --seed {seed} -b {batchsize} -stp {step_interval} {mcr} {spec_loss} --lambda_spec {lambda_spec} -lr {learn_rate} --nodes {nodes} --procs {procs} --suffix {job_name} --epochs {epochs} 

echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [18]:
# bash_strg

In [19]:
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'

In [20]:
fname='batch_train.sh'
filename=staging_loc+fname
with open (filename,'w') as f:
    f.write(bash_strg)

In [21]:
print(filename)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/batch_train.sh


In [22]:
### Move to staging locations in project space:
os.chdir(staging_loc)

In [23]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8 
#SBATCH --cpus-per-task=8 
#SBATCH --gpus-per-task=1
#SBATCH --time=2:00:00
#SBATCH --job-name=scale0.1_deterministic_off

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

### Initial setup
module purge
module load cgpu
module load modules/3.2.11.4 gcc/8.3.0 cuda/10.2.89 mvapich2/2.3.2 cmake/3.14.4 python3/3.7-anaconda-2019.10

module use /global/cfs/cdirs/m3363/lbann/tom_lbann_install_20210126/etc/modulefiles
module load lbann

export MKL_THREADING_LAYER=GNU

#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git
export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/tom_spack
$SPACK_ROOT/share/spack/setup-env.sh
export MV2_ENABLE_AFFINITY=0
export MV2_USE_CUDA=1
export IBV_FORK_SAFE=1

### Run the main code
code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lba

## Submit job to cori GPU

In [24]:
%%bash -s "$filename" ## Use python variable in bash
module load cgpu
sbatch $1

Submitted batch job 1440579
